In [2]:
import ray.data
import datasets

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model = AutoModelForCausalLM.from_pretrained("gpt2").to("cuda")
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [7]:
hf_dataset = datasets.load_dataset("Dahoas/rm-static")
ray_dataset = ray.data.from_huggingface(hf_dataset)

Using custom data configuration default-07f3a84fc559881b
Reusing dataset parquet (/home/ray/.cache/huggingface/datasets/parquet/default-07f3a84fc559881b/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/2 [00:00<?, ?it/s]

find: ‘.git’: No such file or directory
[2023-04-13 16:03:19]  INFO ray._private.worker::Connecting to existing Ray cluster at address: 10.0.21.207:6379...
[2023-04-13 16:03:20]  INFO ray._private.worker::Connected to Ray cluster. View the dashboard at https://console.anyscale-staging.com/api/v2/sessions/ses_46353qtdd4jn1lx3d4sevlg2a8/services?redirect_to=dashboard 
[2023-04-13 16:03:20]  INFO ray._private.runtime_env.packaging::Pushing file package 'gcs://_ray_pkg_006bf5d5dd5d9ec7d200ab30b6e9978f.zip' (0.02MiB) to Ray cluster...
[2023-04-13 16:03:20]  INFO ray._private.runtime_env.packaging::Successfully pushed file package 'gcs://_ray_pkg_006bf5d5dd5d9ec7d200ab30b6e9978f.zip'.


In [8]:
raw_record = ray_dataset["train"].take(1)[0]
train_input = raw_record["prompt"] + raw_record["chosen"]


print(repr(train_input))


output: 0 queued: 100%|██████████| 1/1 [00:00<00:00, 395.47it/s]
                                                                

'\n\nHuman: Can you describe the steps to clean fingerprints and smudges from a laptop screen\n\nAssistant: Yes, certainly. To clean your screen, you first need to use a microfiber cloth or soft, damp cloth to gently wipe down the surface of the screen. Next, you’ll want to grab a soft, lint-free, microfiber cleaning cloth and gently rub it back and forth across the screen to remove fingerprints and smudges.\n\nHuman: Can I spray isopropyl alcohol onto the cloth and clean it that way?\n\nAssistant: Yes, you can do that to help the cloth pick up even more dirt from the screen. Be sure to always use a clean, soft cloth, not a piece of scratchy, roughened, or textured material, and make sure it’s lint-free.'


In [52]:
tokenizer = AutoTokenizer.from_pretrained(
    "gpt2", 
    use_fast=True,
    padding_side="left", # You want the prompt to be at the end of the input
    truncation_side="left"
)
block_size = 512


In [65]:
print("len size in tokenizer: ", len(tokenizer))
print(model.get_input_embeddings())
tokenizer.pad_token = tokenizer.eos_token
print("len size in tokenizer after padding: ", len(tokenizer))
model.config.end_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id
print("configs are setup for model too")
model.resize_token_embeddings(len(tokenizer))
print(model.get_input_embeddings())

len size in tokenizer:  50257
Embedding(50257, 768)
len size in tokenizer after padding:  50257
configs are setup for model too
Embedding(50257, 768)


In [95]:
tokenizer.pad_token = tokenizer.eos_token
input_texts = [train_input, "foo"]
train_tensors = tokenizer(
    input_texts,
    truncation=True,
    max_length=block_size,
    padding="max_length",
    return_tensors="pt", 
)
tokenized_texts = [tokenizer.decode(train_tensors["input_ids"][i]) for i in range(len(train_tensors["input_ids"]))]
print(tokenized_texts)
print("-"*80)
for input_text in input_texts:
    print(input_text)
print("tokenize text shape")
print(train_tensors["input_ids"].shape)

['<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

In [10]:
tokenizer.model_max_length

1024

In [16]:
train_tensors["attention_mask"]

tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 1]])

In [ ]:
tokenizer.eos_token

'<|endoftext|>'

In [70]:
output = model.generate(
    input_ids=train_tensors["input_ids"].to("cuda"),
    attention_mask=train_tensors["attention_mask"].to("cuda"),
    max_new_tokens=512,  
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id, 
)


In [71]:
output_text = tokenizer.batch_decode(output, skip_special_tokens=True)

In [72]:
output_text

['\n\nHuman: Can you describe the steps to clean fingerprints and smudges from a laptop screen\n\nAssistant: Yes, certainly. To clean your screen, you first need to use a microfiber cloth or soft, damp cloth to gently wipe down the surface of the screen. Next, you’ll want to grab a soft, lint-free, microfiber cleaning cloth and gently rub it back and forth across the screen to remove fingerprints and smudges.\n\nHuman: Can I spray isopropyl alcohol onto the cloth and clean it that way?\n\nAssistant: Yes, you can do that to help the cloth pick up even more dirt from the screen. Be sure to always use a clean, soft cloth, not a piece of scratchy, roughened, or textured material, and make sure it’s lint-free.\n\nHuman: Can I use a soft, lint-free, microfiber cleaning cloth to clean my laptop screen?\n\nAssistant: Yes, you can use a soft, lint-free, microfiber cleaning cloth to clean your laptop screen.\n\nHuman: Can I use a soft, lint-free, microfiber cleaning cloth to clean my laptop scre

In [103]:
from transformers import pipeline
generator = pipeline("text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        device=0)

output_text = generator(input_texts[0], max_new_tokens=512, do_sample=True, top_k=50, top_p=0.95, num_return_sequences=2)


In [104]:
output_text[0].keys()

2

In [101]:
foo = train_input[0] + output_text[0]["generated_text"]
print(foo)




Human: Can you describe the steps to clean fingerprints and smudges from a laptop screen

Assistant: Yes, certainly. To clean your screen, you first need to use a microfiber cloth or soft, damp cloth to gently wipe down the surface of the screen. Next, you’ll want to grab a soft, lint-free, microfiber cleaning cloth and gently rub it back and forth across the screen to remove fingerprints and smudges.

Human: Can I spray isopropyl alcohol onto the cloth and clean it that way?

Assistant: Yes, you can do that to help the cloth pick up even more dirt from the screen. Be sure to always use a clean, soft cloth, not a piece of scratchy, roughened, or textured material, and make sure it’s lint-free.

Human: Can I clean my laptop by pressing a button on the screen?

Assistant: Yes, you can press a button in the middle of a screen, but you will need to hold down the power button while pressing it to be able to use the screen. Also, press the power button when touching a computer or USB, if 

In [102]:
print(tokenizer(foo, return_tensors="pt").input_ids.shape)

torch.Size([1, 694])


In [29]:
for input_id, attn_mask in zip(train_tensors["input_ids"], train_tensors["attention_mask"]):
    print(input_id)
    output = model.generate(
        input_ids=input_id[None].to("cuda"),
        attention_mask=attn_mask[None].to("cuda"),
        max_new_tokens=1024
    )
    print(output)

tensor([ 2705, 16270,    11,   407,   257,  3704,   286, 12692,    88,    11,
        13805,    70,   831,   276,    11,   393,  2420,  1522,  2587,    11,
          290,   787,  1654,   340,   447,   247,    82,   300,   600,    12,
         5787,    13])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [5,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [5,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [5,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [5,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [5,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1088: indexSelectSmallIndex: block: [5,0,0], thread: [69,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/nativ

RuntimeError: The size of tensor a (1024) must match the size of tensor b (1025) at non-singleton dimension 3

In [ ]:
outputs.shape

torch.Size([2, 1025])

In [ ]:
import torch
assert torch.allclose(outputs[:, block_size:].to("cpu"), train_tensors["input_ids"])

AssertionError: 

In [1]:
from transformers import AutoModelForSequenceClassification
from optimum.bettertransformer import BetterTransformer
model_hf = AutoModelForSequenceClassification.from_pretrained("facebook/opt-13b")
model = BetterTransformer.transform(model_hf, keep_original_model=False)

/home/ray/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [3]:
model.device

device(type='cpu')